In [ ]:
!pip install -q transformers==4.37.2
!wget -O scibert_uncased.tar https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
!tar -xvf scibert_uncased.tar
!pip install sentence_transformers
import torch
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from tqdm import tqdm
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 32.8 MB/s eta 0:00:00
--2024-04-07 02:23:29--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.246.104, 52.92.162.200, 52.92.184.160, ...
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.246.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 442460160 (422M) [application/x-tar]
Saving to: ‘scibert_uncased.tar’

scibert_uncased.tar 100%[===================>] 421.96M  17.1MB/s    in 24s     

2024-04-07 02:23:54 (17.5 MB/s) - ‘scibert_uncased.tar’ saved [442460160/442460160]

scibert_scivocab_uncased/
scibert_scivocab_uncased/vocab.txt
scibert_scivocab_uncased/pytorch_model.bin
scibert_scivocab_uncased/config.json
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7

In [ ]:
#reading features dataset
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/w266_project')
df = pd.read_csv(f"/content/drive/My Drive/Colab Notebooks/w266_project/join_df.csv", header = 0, sep="\t", index_col=0)

Mounted at /content/drive


In [ ]:
# Load SciBERT tokenizer and model
sci_bert_tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased", do_lower_case=True)
sci_bert = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Base Line Model


In [ ]:

# Load SciBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")




# Tokenize abstracts and generate SciBERT embeddings
def tokenize_and_embed(text):
    tokens = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)
    input_ids = torch.tensor([tokens])
    with torch.no_grad():
        outputs = model(input_ids)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()  # Mean pooling of token embeddings
    return embeddings
df['embeddings'] = df['Abstract'].apply(tokenize_and_embed)

# Split dataset into train and test sets
X = np.vstack(df['embeddings'])
y = df['JIF']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train linear regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Evaluate model using Pearson correlation coefficient
train_predictions = regressor.predict(X_train)
val_predictions = regressor.predict(X_val)

train_corr, _ = pearsonr(y_train, train_predictions)
val_corr,_ = pearsonr(y_val, val_predictions)

print(f"Train Pearson correlation coefficient: {train_corr:.4f}")
print(f"Validation Pearson correlation coefficient: {val_corr:.4f}")


Train Pearson correlation coefficient: 0.6957
Validation Pearson correlation coefficient: 0.5150
